### Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest

### Configs

In [2]:
DATA_PATH = '/Users/mpekey/Desktop/FlyVideo/Peak_Signal_Data'

FEATURES = ['pose.prob_x','pose.prob_y','pose.halt_x','pose.halt_y',
            'pose.thor_post_x','pose.thor_post_y','distance.origin-halt',
            'distance.origin-prob','distance.origin-thor_post','distance.head-prob',
            'distance.thor_post-halt','distance.avg(thor_post-joint1,thor_post-joint2,thor_post-joint3)',
            'distance.avg(origin-joint1,origin-joint2,origin-joint3)']

experiment_features = ['pose.prob_x', 'pose.prob_y', 'distance.head-prob', 'distance.origin-prob']

### Read Data

In [3]:
filename = os.path.join(DATA_PATH, 'bouts_dict.pkl')
with open(filename, 'rb') as f:
    bouts_dict = pickle.load(f)

true_peak_fn = os.path.join(DATA_PATH, 'true_peak_annotations.npy')
true_peak_df_fn = os.path.join(DATA_PATH, 'true_annotations.pkl')

true_peak_annotations_array = np.load(true_peak_fn)
with open(true_peak_df_fn, 'rb') as f:
    true_peak_annotations_df = pickle.load(f)

### Utility Functions

In [4]:
def create_input_data(fly, experiment, features):
    
    input_data = bouts_dict[fly][features[0]][experiment].reshape(-1,1)

    for i in range(1, len(features)):
        input_data = np.concatenate((input_data,
                                    bouts_dict[fly][features[i]][experiment].reshape(-1,1)),
                                    axis=1)
    return input_data

In [59]:
from sklearn.metrics import classification_report, confusion_matrix

def calculate_metrics(predictions, labels):
    TP = sum(p == 1 and l == 1 for p, l in zip(predictions, labels))
    TN = sum(p == 0 and l == 0 for p, l in zip(predictions, labels))
    FP = sum(p == 1 and l == 0 for p, l in zip(predictions, labels))
    FN = sum(p == 0 and l == 1 for p, l in zip(predictions, labels))
    
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    
    if TP + FN > 0:
        recall = TP / (TP + FN)
    else:
        recall = 0.0
    
    if TP + FP > 0:
        precision = TP / (TP + FP)
    else:
        precision = 0.0
    
    if precision + recall > 0:
        f1_score = 2 * (precision * recall) / (precision + recall)
    else:
        f1_score = 0.0
    
    return accuracy, recall, precision, f1_score, {'TP':TP,'TN':TN,'FP':FP,'FN':FN}


def calculate_grouped_recall(predicted_peaks, true_peaks, matching_range):
    recall_predictions = []

    for true_idx in true_peaks:
        found_true_pred = False
        for pred_idx in predicted_peaks:
            if abs(pred_idx - true_idx) <= matching_range:
                found_true_pred = True
                break
        recall_predictions.append(found_true_pred)
    

    recall = np.sum(recall_predictions) / len(recall_predictions)
    return recall


def evaluate_classification(predictions, labels):
    conf_matrix = confusion_matrix(labels, predictions)
    class_report = classification_report(labels, predictions, target_names=["Normal", "Anomaly"])
    return conf_matrix, class_report


def create_evaluation_lists(true_data, predicted_data, data_length, matching_range):
    predictions = [0] * data_length
    for p in predicted_data:
        predictions[p] = 1

    labels = [0] * data_length
    for p in true_data:
        for idx in range(p - matching_range, p + matching_range):
            if idx >= 0 and idx < data_length:
                labels[idx] = 1
    return predictions, labels


def calculate_evaluation_metrics(data_length, predicted_peaks, true_peaks, matching_range):
    predictions, labels = create_evaluation_lists(true_peaks, predicted_peaks, data_length, matching_range)
    accuracy, recall, precision, f1_score, conf_mat_metrics = calculate_metrics(predictions, labels)
    grouped_recall = calculate_grouped_recall(predicted_peaks, true_peaks, matching_range)
    return accuracy, recall, precision, f1_score, grouped_recall, conf_mat_metrics

def evaluate_results(all_results, matching_range = 30):
    avg_recall, avg_precision, avg_accuracy, avg_f1_score = 0, 0, 0, 0
    all_recall, all_precision, all_accuracy, all_f1_score = [], [], [], []
    
    # If any prediction matches
    avg_grouped_recall = 0
    all_grouped_recall, all_conf_mat_metrics = [], []

    for res in all_results:
        predicted_peaks = res['predicted_index']
        true_peaks = res['true_index']
        data_length = res['data_length']
        
        accuracy, recall, precision, f1_score, grouped_recall, conf_mat_metrics = calculate_evaluation_metrics(data_length,
                                                                                                               predicted_peaks,
                                                                                                               true_peaks,
                                                                                                               matching_range)
        all_recall.append(recall)
        all_precision.append(precision)
        all_accuracy.append(accuracy)
        all_f1_score.append(f1_score)
        all_grouped_recall.append(grouped_recall)
        all_conf_mat_metrics.append(conf_mat_metrics)
        
        avg_recall += recall
        avg_precision += precision
        avg_accuracy += accuracy
        avg_f1_score += precision
        avg_grouped_recall += grouped_recall

    avg_recall /= len(all_results)
    avg_precision /= len(all_results)
    avg_accuracy /= len(all_results)
    avg_f1_score /= len(all_results)
    avg_grouped_recall /= len(all_results)
    
    return {'avg_metrics' : {'avg_accuracy' : avg_accuracy,
                             'avg_precision' : avg_precision,
                             'avg_recall' : avg_recall,
                             'avg_f1_score' : avg_f1_score,
                             'avg_grouped_recall' : avg_grouped_recall},
            'all_metrics' : {'all_accuracy' : all_accuracy,
                             'all_precision' : all_precision,
                             'all_recall' : all_recall,
                             'all_f1_score' : all_f1_score,
                             'all_grouped_recall' : all_grouped_recall},
            'conf_mat_metrics' : all_conf_mat_metrics}

In [6]:
def filter_prediction(anomalies, grouped_range=60):
    idx_group = []
    all_groups = []
    for idx in list(anomalies.index):
        if idx in idx_group:
            continue
        idx_group = []
        for i in range(idx, idx+grouped_range):
            if i in list(anomalies.index):
                idx_group.append(i)
        all_groups.append(idx_group)
    
    group_pred_idx = []
    group_pred_val = []
    for group in all_groups:
        group_vals = anomalies.loc[group]['distance.origin-prob']
        if len(group_vals) > 1 and group_vals.iloc[0] < group_vals.iloc[1]:
            pred = group_vals[group_vals == group_vals.max()]
        else:
            pred = group_vals[group_vals == group_vals.min()]
        group_pred_idx.append(pred.index.values[0])
        group_pred_val.append(pred.values[0])
    return group_pred_idx, group_pred_val

### Fly Class

In [7]:
class FlyInfo:
    def __init__(self, name, trial_id, peak_index, peak_values):
        self.name = name
        self.trial_id = trial_id
        self.peak_index = peak_index
        self.peak_values = peak_values

class FlyDatabase:
    def __init__(self):
        self.fly_data = []

    def add_fly(self, fly_info):
        self.fly_data.append(fly_info)

    def get_fly(self, name, trial_id):
        for fly_info in self.fly_data:
            if fly_info.name == name and fly_info.trial_id == trial_id:
                return fly_info
        return None
    
    def write_fly_info(self, name, trial_id):
        for fly_info in self.fly_data:
            if fly_info.name == name and fly_info.trial_id == trial_id:
                print('Name:', fly_info.name)
                print('Trial Id:', fly_info.trial_id)
                print('Peak Index:', fly_info.peak_index)
                print('Peak Values:', fly_info.peak_values)
                return None
        print('Fly not found!!!')
        return None

In [8]:
fly_db = FlyDatabase()

fly_names = true_peak_annotations_df['name'].unique()

for name in fly_names:
    trial_idxs = true_peak_annotations_df[true_peak_annotations_df['name'] == name]['trial_id'].unique().tolist()
    for idx in trial_idxs:
        peak_index = true_peak_annotations_df[(true_peak_annotations_df['name'] == name) & (true_peak_annotations_df['trial_id'] == idx)]['peak_index'].values
        peak_values = true_peak_annotations_df[(true_peak_annotations_df['name'] == name) & (true_peak_annotations_df['trial_id'] == idx)]['value'].values
        fly_db.add_fly(FlyInfo(name, idx, peak_index, peak_values))

fly_db.write_fly_info('Fly05182022_5d', '0')

Name: Fly05182022_5d
Trial Id: 0
Peak Index: [ 534  694  903 1207 1623]
Peak Values: [1014.36112044 1017.73376888 1027.22816309 1021.72159098 1023.54593544]


In [68]:
peak_ratios = []
for fly in fly_db.fly_data:
    data_length = len(bouts_dict[fly.name].loc[int(fly.trial_id)]['distance.origin-prob'])
    peak_amount = len(fly.peak_index) * 60
    peak_ratios.append(peak_amount / data_length)

peak_ratio = np.mean(peak_ratios)
max_ratio = np.max(peak_ratios)
min_ratio = np.min(peak_ratios)

print(f'Ratio: {peak_ratio}, Min: {min_ratio}, Max: {max_ratio}')

Ratio: 0.20264069768278617, Min: 0.018484288354898338, Max: 0.6469002695417789


### Evaluation

In [60]:
model_features = ['distance.origin-prob',
                  'distance.head-prob',
                  'pose.prob_x',
                  'pose.prob_y']

all_results = []
all_results_group = []

for fly in fly_db.fly_data:
    input_data = create_input_data(fly = fly.name,
                                   experiment = int(fly.trial_id),
                                   features = model_features)
    
    info_df = pd.DataFrame(input_data, columns = model_features)
    scaled_data = StandardScaler().fit_transform(input_data)
    model = IsolationForest(n_estimators = 100,
                            contamination=0.04,
                            max_samples='auto',
                            max_features=1.0,
                            bootstrap=False)
    info_df['predictions'] = model.fit_predict(scaled_data)
    anomalies = info_df.loc[info_df['predictions'] == -1, ['distance.origin-prob']]
    anomalies_idx = list(info_df.loc[info_df['predictions'] == -1, ['distance.origin-prob']].index)

    group_pred_idx, group_pred_val = filter_prediction(anomalies, grouped_range=60)
    
    all_results.append({'true_index': fly.peak_index, 'predicted_index': anomalies_idx, 'data_length': len(info_df['predictions'].values)})
    all_results_group.append({'true_index': fly.peak_index, 'predicted_index': group_pred_idx, 'data_length': len(info_df['predictions'].values)})

results = evaluate_results(all_results)
results_g = evaluate_results(all_results_group)

In [62]:
results['avg_metrics']

{'avg_accuracy': 0.8193176491316547,
 'avg_precision': 0.7621257926868178,
 'avg_recall': 0.19843957468929802,
 'avg_f1_score': 0.7621257926868178,
 'avg_grouped_recall': 0.8214927332968859}

In [63]:
results_g['avg_metrics']

{'avg_accuracy': 0.7999980275830455,
 'avg_precision': 0.7557723485037828,
 'avg_recall': 0.013715486453742442,
 'avg_f1_score': 0.7557723485037828,
 'avg_grouped_recall': 0.8030107324650999}

### Deneme Alani

In [48]:
model_features = ['distance.origin-prob',
                  'distance.head-prob',
                  'pose.prob_x',
                  'pose.prob_y']

fly = fly_db.fly_data[0]

input_data = create_input_data(fly = fly.name,
                                   experiment = int(fly.trial_id),
                                   features = model_features)
    
info_df = pd.DataFrame(input_data, columns = model_features)
scaled_data = StandardScaler().fit_transform(input_data)
model = IsolationForest(n_estimators = 100,
                        contamination=0.04,
                        max_samples='auto',
                        max_features=1.0,
                        bootstrap=False)
info_df['predictions'] = model.fit_predict(scaled_data)
anomalies = info_df.loc[info_df['predictions'] == -1, ['distance.origin-prob']]
anomalies_idx = list(info_df.loc[info_df['predictions'] == -1, ['distance.origin-prob']].index)

group_pred_idx, group_pred_val = filter_prediction(anomalies, grouped_range=60)

In [52]:
results = evaluate_results([{'true_index': fly.peak_index,
                            'predicted_index': anomalies_idx,
                            'all_preds': info_df['predictions'].values.tolist()}])

TP 43 TN 1818 FP 44 FN 257


In [53]:
results['avg_metrics']

{'avg_accuracy': 0.8607770582793709,
 'avg_precision': 0.4942528735632184,
 'avg_recall': 0.14333333333333334,
 'avg_f1_score': 0.4942528735632184,
 'avg_grouped_recall': 1.0}

In [54]:
results_g = evaluate_results([{'true_index': fly.peak_index,
                            'predicted_index': group_pred_idx,
                            'all_preds': info_df['predictions'].values.tolist()}])

TP 5 TN 1860 FP 2 FN 295


In [55]:
results_g['avg_metrics']

{'avg_accuracy': 0.862627197039778,
 'avg_precision': 0.7142857142857143,
 'avg_recall': 0.016666666666666666,
 'avg_f1_score': 0.7142857142857143,
 'avg_grouped_recall': 1.0}